In [1]:
import numpy as np
from pathlib import Path
import utils
import mne


a:\Users\Szymon Mazurek\anaconda3\envs\sano_eeg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data_preprocessed = mne.io.read_raw_edf(Path('preprocessed_data/chb21/chb21_19.edf'), preload=False, verbose=False)
input_array = data_preprocessed.get_data()

In [3]:
def prepare_timestep_array(array, timestep, overlap):
    """Preprocess input array of shape [n_nodes,feature_per_node,samples]
    into [samples_count,n_nodes,feature_per_node,timestep]."""
    features = [
        array[:, :, i : i + timestep]
        for i in range(0, array.shape[2] - timestep + 1, timestep - overlap)
    ]
    return np.array(features)


def prepare_timestep_label(array, timestep, overlap):
    """Preprocess input array of shape [n_nodes,feature_per_node,samples]
    into [samples_count,n_nodes,feature_per_node,timestep]."""
    time_to_seizure = array.shape[2]
    seconds = [
        (time_to_seizure - i) / 256
        for i in range(timestep, array.shape[2], timestep - overlap)
    ]
    return np.array(seconds)

In [9]:
start_ev = np.array([1288,2000])
end_ev = np.array([1344,2100])


In [42]:

features,labels,time_labels = extract_training_data_and_labels_array(input_array,start_ev,end_ev,overlap=0)

In [ ]:
adult_data =Path('raw_dataset')
peprocessed_data = Path('preprocessed_data')
subject_seizures = Path('raw_dataset/RECORDS-WITH-SEIZURES')

In [ ]:
utils.preprocess_dataset(subject_seizures,adult_data,peprocessed_data)

In [ ]:
data_raw =utils.load_and_dump_channels(Path('raw_dataset/chb01/chb01_03.edf'))
utils.reorder_channels_chbmit(data_raw)
raw_copy = utils.run_preprocessing(data_raw,18,apply_ica=False,apply_pca = True,freq_l=2)